# Computer Vision Project
## Team 1
  - AHyun Ji
  - Eunsang Lee
  - Song Yunjin


In [ ]:
# 라이브러리 import
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.io as sio

from PIL import Image
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.io import imread, imsave
from scipy.io import loadmat
from tqdm import tqdm

In [ ]:
# 이미지 불러오기, 수정
!wget -O CV-term-prj.zip https://github.com/twoSilverUp/CV-term-prj/archive/refs/heads/main.zip

import zipfile
with zipfile.ZipFile("CV-term-prj.zip", 'r') as zip_ref:
    zip_ref.extractall("CV-term-prj")

with zipfile.ZipFile("CV-term-prj/CV-term-prj-main/train.zip", 'r') as zip_ref:
    zip_ref.extractall("project_data/")

with zipfile.ZipFile("CV-term-prj/CV-term-prj-main/test.zip", 'r') as zip_ref:
    zip_ref.extractall("project_data/")

--2024-12-07 18:11:11--  https://github.com/twoSilverUp/CV-term-prj/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/twoSilverUp/CV-term-prj/zip/refs/heads/main [following]
--2024-12-07 18:11:11--  https://codeload.github.com/twoSilverUp/CV-term-prj/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘CV-term-prj.zip’

CV-term-prj.zip         [  <=>               ]  10.25M  29.8MB/s    in 0.3s    

2024-12-07 18:11:12 (29.8 MB/s) - ‘CV-term-prj.zip’ saved [10749694]



## Create npy file

In [ ]:
dir_path = '/content/project_data/test'
jpg_files = [file for file in os.listdir(dir_path) if file.endswith('.jpg')]

In [ ]:
save_dir_path = '/content/save'
os.makedirs(save_dir_path, exist_ok=True)

### edge detection function

In [ ]:
import cv2

def make_image(image_path):

    image = cv2.imread(image_path)

    # image preprocessing
    alpha1 = - 0.1

    image = np.clip((1 + alpha1) * image - 128 * alpha1, 0, 255).astype(np.uint8)
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # sobel
    dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(dx, dy)
    sobel_combined = np.clip(sobel_combined, 0, 255).astype(np.uint8)


    # binarization
    median_ = np.median(sobel_combined)
    l = int(min(median_ + 100, 180))

    _, binary_edges = cv2.threshold(sobel_combined, l, 255, cv2.THRESH_BINARY)

    binary_edges = cv2.cvtColor(binary_edges, cv2.COLOR_BGR2GRAY)

    # connected components filtering
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_edges, connectivity=8)
    min_area = 20

    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] < min_area:
            binary_edges[labels == i] = 0

    # window filtering
    brightness_threshold = 200
    window_size = 31
    half_window = window_size // 2

    not_boundary = []
    for i in range(binary_edges.shape[0]):
        for j in range(binary_edges.shape[1]):
            x_start = max(i - half_window, 0)
            x_end = min(i + half_window + 1, binary_edges.shape[0])
            y_start = max(j - half_window, 0)
            y_end = min(j + half_window + 1, binary_edges.shape[1])

            window = binary_edges[x_start:x_end, y_start:y_end]
            num_bright_neighbors = np.sum(window >= brightness_threshold)

            if num_bright_neighbors > window_size * window_size // 5:
                not_boundary.append((i, j))

    for i, j in not_boundary:
          binary_edges[i, j] = 0

    # connected components filtering
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_edges, connectivity=8)
    min_area = 50

    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] < min_area:
            binary_edges[labels == i] = 0

    # Gaussian blur
    binary_edges = cv2.GaussianBlur(binary_edges, (11, 11), 7)

    # cv2_imshow(binary_edges)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return binary_edges

In [ ]:
for jpg_file in jpg_files:
    image_path = os.path.join(dir_path, jpg_file)
    file_name = os.path.splitext(jpg_file)[0]

    output = make_image(image_path)

    save_path = os.path.join(save_dir_path, f"{file_name}.jpg")
    cv2.imwrite(save_path, output)

In [ ]:
save_path = "/content/save"  # image path
npy_path = "/content/predictions.npy"  # npy path


image_files = [f for f in os.listdir(save_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

image_dict = {}

for file in image_files:
    img_path = os.path.join(save_path, file)
    img = Image.open(img_path)
    img_array = np.array(img, dtype=np.float32)


    key = os.path.splitext(file)[0]
    image_dict[key] = img_array

np.save(npy_path, image_dict)

print(npy_path)

/content/predictions.npy
